In [1]:
import h5py
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm

In [2]:
classes = ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK',
           '16APSK', '32APSK', '64APSK', '128APSK', '16QAM', '32QAM', '64QAM',
           '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC',
           'AM-DSB-SC', 'FM', 'GMSK', 'OQPSK']

class_subset = [3, 5, 13, 20, 21]  # BPSK, 8PSK, 64QAM, AM-DSB-SC, FM

In [3]:
file_path = r"D:\datasets\Radio_Spectrum\RadioML 2018.01A\GOLD_XYZ_OSC.0001_1024.hdf5"

with h5py.File(file_path, 'r') as f:
    y = pd.DataFrame(f['Y'])
y

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555899,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2555900,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2555901,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2555902,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
mask = y[class_subset].sum(axis=1) == 1
filtered = y[mask]
filtered = filtered[class_subset]


In [5]:
filtered

,3,5,13,20,21
319488,1,0,0,0,0
319489,1,0,0,0,0
319490,1,0,0,0,0
319491,1,0,0,0,0
319492,1,0,0,0,0
...,...,...,...,...,...
2342907,0,0,0,0,1
2342908,0,0,0,0,1
2342909,0,0,0,0,1
2342910,0,0,0,0,1


In [6]:
index_set = set(filtered.index)

with h5py.File(file_path, 'r') as f:
    X_data = []
    Z_data = []

    for n in tqdm(range(2555899)):
        if n in index_set:
            X_data.append(f['X'][n])
            Z_data.append(f['Z'][n])

print(len(Z_data))
    

    

100%|██████████| 2555899/2555899 [03:12<00:00, 13292.21it/s] 

532480


In [7]:
# Convert to numpy after filtering
X_data = np.array(X_data, dtype=np.float16)
y_labels = filtered.idxmax(axis=1).to_numpy()
Z_data = np.array(Z_data).reshape(-1)

In [9]:
dataset = {
    'X': np.array(X_data),
    'y': y_labels,      
    'snr': np.array(Z_data).reshape(-1)
}

In [10]:
# save dataset
with open('subset.pkl', 'wb') as file:
    pickle.dump(dataset, file)